In [1]:
import basedosdados as bd
import pandas as pd

## Seleção do período:
- desde 2022-01-01, início do atual mandato da câmara de deputados
- até 2024-04-01, limite do dataset com acesso gratuito 

# Observações

### Passo 1: Análise de texto de leis propostas

Cada linha da base de dados corresponde a uma proposição, que pode ser de variados tipos.
Os que parecem ser mais relevantes são "Projeto de Lei" ou "Redação Final"
A escolha depende de (1) quanto detalhe se quer ter sobre o texto da lei e (2) o quão próximo o texto é da lei que foi votada e talvez aprovada.
- (1) Se apenas a emenda (descrição da proposição) for suficiente, dá pra usar no bag of words a coluna "ementa" das proposições tipo "Projeto de Lei". Se quiser mais detalhe, é melhor usar o texto completo da lei, coluna "teor_completo", das proposições tipo "Redação Final".
- (2) Se quiser o projeto de lei da forma que foi proposto pelo parlamentar, é melhor usar o tipo "Projeto de Lei", se quiser a lei como ela foi talvez aprovada ou votada, é melhor usar "Redação Final"

Caso a escolha seja usar o texto completo ("teor_completo") da "Redação Final", é preciso encontrar um library que faça a leitura do PDF, e resolver o erro que tive. Também vai ter uma lmiitação de performance por armazenar o texto da web na memória.

### Passo 2: Correlação com temas das leis propostas e sua aprovação

Essa base de dados tem outras tabelas relacionadas à votação, também com o ID da proposição. Mas precisa investigar mais pra entender qual proposição representa a votação do PL antes de passar à sanção do presidente da república (último passo).



## Seleção do tipo de proposição: 'Projeto de Lei' ou 'Redação Final' 

In [2]:
# SQL query

list_types = '''
SELECT DISTINCT
  sigla,
  tipo,
FROM `basedosdados.br_camara_dados_abertos.proposicao_microdados` AS dados
WHERE (dados.data > '2022-01-01' AND dados.data < '2024-04-01')
'''

# Read query via 'Base dos dados' Google Cloud API
projects_list = bd.read_sql(list_types, billing_project_id='anpec-base-dos-dados')

print('**LISTA DOS DIVERSOS TIPOS DE PROPOSIÇÃO**')
for n in projects_list['tipo']:
    print(n)

Downloading: 100%|████████████████████████████████████████████████████████████████| 235/235 [00:00<00:00, 732.26rows/s]

**LISTA DOS DIVERSOS TIPOS DE PROPOSIÇÃO**
Mensagem
Substitutivo
Projeto de Lei
Representação
Processo Interno
Emenda de Relator
Ofício do Senado Federal
Relatório do Relator (CMO)
Subemenda Adotada pela Comissão
Requerimento de Inversão de Pauta
Destaque para Votação em Separado
Proposta de Emenda à Constituição
Requerimento de Instituição de CPI
Requerimento de Adiamento da Discussão
Emenda de Plenário a Projeto com Urgência
Requerimento de Urgência (Art. 155 do RICD)
Requerimento de Encerramento de Discussão em Comissão
Requerimento de Audiência solicitada por Comissão ou Deputado
Requerimento para envio de Requerimento de Informação pela Comissão
Aviso (CN) de Relatório de Gestão Fiscal do Tribunal de Contas da União
Documento
Emenda à PEC
Relatório Final
Relatório Preliminar
Objeto de Deliberação
Perempção de Rádio/TV
Requerimento de Preferência
Instalação de Grupo Parlamentar
Sugestão de Emenda ao PPA - Comissões
Projeto de Lei (CN) de Alteração da LOA
Requerimento de Convocação 

- [Projeto de Lei](https://www.congressonacional.leg.br/legislacao-e-publicacoes/glossario-legislativo/-/legislativo/termo/projeto_de_lei_pl) - Proposição de uma lei ordinária, vai ser sancionado (aprovado) pelo presidente da república
- [Projeto de Lei Complementar](https://www.congressonacional.leg.br/legislacao-e-publicacoes/glossario-legislativo/-/legislativo/termo/lei_complementar_lc) - adiciona detalhes a uma lei já aprovada
- [Redação Final](https://www.congressonacional.leg.br/legislacao-e-publicacoes/glossario-legislativo/-/legislativo/termo/redacao_final) - texto do projeto de lei a ser votado pelo plenário da câmara

  **Projeto de lei é a lei quando recém proposta, ainda precisando ser votada pela plenária da câmara e senado (quando se aplica). Redação final é o projeto já ajustado pela Câmara e Senado, antes de ir para sanção (aprovação) do Presidente da República**

[Processo legislativo](https://www.camara.leg.br/entenda-o-processo-legislativo/)

## Detalhamento do tipo 'Projeto de Lei'

In [25]:
# SQL query

query_projects = '''
SELECT DISTINCT
  sigla,
  tipo,
  ementa,
  ementa_detalhada,
  palavra_chave,
  situacao_ultimo_status,
  apreciacao_ultimo_status,
  url_teor_proposicao
FROM `basedosdados.br_camara_dados_abertos.proposicao_microdados` AS dados
WHERE (dados.data > '2022-01-01' AND dados.data < '2024-04-01')
AND dados.tipo = 'Projeto de Lei'
'''

# Read query via 'Base dos dados' Google Cloud API
projects = bd.read_sql(query_projects, billing_project_id='anpec-base-dos-dados')

Downloading: 100%|█████████████████████████████████████████████████████████████| 8904/8904 [00:06<00:00, 1453.49rows/s]


In [5]:
# contagem de projetos de lei no período

projects[projects['tipo'] == "Projeto de Lei"]['tipo'].value_counts()

Projeto de Lei    8897
Name: tipo, dtype: int64

In [60]:
# campo de emenda é a descrição do projeto de lei

for n in projects['ementa'][:5]:
    print(n)

Altera a Lei nº 9.503, de 23 de setembro de 1997, que institui o Código de Trânsito Brasileiro, para determinar que os      micro-ônibus e ônibus empregados nos serviços de transporte coletivo de passageiros possuam botão de pânico.
Denomina “João Bosco de Aquino Araújo” o trecho rodoviário no Km 468,6 da BR-070 (GO) até o Km 6,7 da BR-070 (MT), no Contorno Rodoviário de Aragarças (GO), Pontal do Araguaia (MT) e Barra do Garças (MT).
Dispõe sobre a prestação de serviços médicos
Dispõe sobre a obrigatoriedade de afixação de cartazes, adesivos ou placas informativas em idiomas distintos nos locais de acesso e permanência de turistas público ou privado, contendo as informações de utilidade e segurança pública aos turistas nos Estados do Brasil, assim como, informações sobre os meios de denúncias, contato e endereço com localização da Delegacia de Proteção ao Turista.
Altera a Lei nº 14.597, de 14 de junho de 2023, que institui a Lei Geral do Esporte, para dispor sobre a aplicabilidade das

In [61]:
# a própria base de dados já tem um label com palavras chaves do projeto de lei

for n in projects['palavra_chave'][:5]:
    print(n)

Alteração, Código de Trânsito Brasileiro, equipamento obrigatório, micro-ônibus, ônibus, transporte coletivo, transporte de passageiro, botão de pânico, segurança pública.
None
Disciplinamento, Prestação de serviços, serviços médicos.
None
None


In [9]:
# status da proposta, não consegui encontrar algo que indique se o projeto de lei foi aprovado

projects['situacao_ultimo_status'].value_counts().head(5)

Tramitando em Conjunto              4338
Aguardando Designação de Relator    1402
Aguardando Parecer                  1235
Pronta para Pauta                    417
Arquivada                            239
Name: situacao_ultimo_status, dtype: int64

In [33]:
# exemplo de ementa

projects['ementa'][1]

'Denomina “João Bosco de Aquino Araújo” o trecho rodoviário no Km 468,6 da BR-070 (GO) até o Km 6,7 da BR-070 (MT), no Contorno Rodoviário de Aragarças (GO), Pontal do Araguaia (MT) e Barra do Garças (MT).'

In [57]:
# exemplo de texto completo do projeto de lei

project_url = projects['url_teor_proposicao'][1]
project_url

'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2382689'

## Detalhamento do tipo 'Redação Final'

In [24]:
# SQL query

query_writing = '''
SELECT DISTINCT
  sigla,
  tipo,
  ementa,
  ementa_detalhada,
  palavra_chave,
  situacao_ultimo_status,
  apreciacao_ultimo_status,
  url_teor_proposicao
FROM `basedosdados.br_camara_dados_abertos.proposicao_microdados` AS dados
WHERE (dados.data > '2022-01-01' AND dados.data < '2024-04-01')
AND dados.tipo = 'Redação Final'
'''

# Read query via 'Base dos dados' Google Cloud API
writing = bd.read_sql(query_writing, billing_project_id='anpec-base-dos-dados')

Downloading: 100%|█████████████████████████████████████████████████████████████| 1559/1559 [00:00<00:00, 2630.57rows/s]


In [12]:
# contagem de redações finais, menos do que projetos de lei, já que ela seria um estágio mais avançad

writing[writing['tipo'] == "Redação Final"]['tipo'].value_counts()

Redação Final    1521
Name: tipo, dtype: int64

In [68]:
# ementa da redação final não é uma descrição tão boa, em muitas linhas só é a referência a um projeto de lei

for n in writing['ementa'][92:97]:
    print(n)

Redação Final do PL 4519/2019
Redação Final do PDL 303/23
Altera a Consolidação das Leis do Trabalho (CLT), aprovada pelo Decreto-Lei nº 5.452, de 1º de maio de 1943, para determinar que as empresas disponibilizem a seus empregados informações sobre campanhas oficiais de vacinação, sobre o papilomavírus humano (HPV) e sobre os cânceres de mama, de colo do útero e de próstata.
Dispõe sobre o direito a indenização por dano moral e a concessão de pensão especial à pessoa com deficiência permanente decorrente de síndrome congênita associada à infecção pelo vírus Zika ou de Síndrome de Guillain-Barré causada pelo vírus Zika, e altera a Consolidação das Leis do Trabalho, aprovada pelo Decreto-Lei nº 5.452, de 1º de maio de 1943, e as Leis nºs 8.742, de 7 de dezembro de 1993, e 8.213, de 24 de julho de 1991.
Redação Final do PDL 284/22


In [14]:
# status da proposta, não consegui encontrar algo que indique se o projeto de lei foi aprovado

writing['situacao_ultimo_status'].value_counts()

Aguardando Providências Internas             396
Pronta para Pauta                             77
Aguardando Apreciação pelo Senado Federal      1
Aguardando Leitura e Publicação                1
Aguardando Recebimento                         1
Name: situacao_ultimo_status, dtype: int64

In [30]:
# Exemplo de ementa

writing['ementa'][1]

'Inscreve o nome de Zilda Arns Neumannno Livro dos Heróis e Heroínas daPátria.'

In [50]:
# Exemplo de texto da redação final

writing_url = writing['url_teor_proposicao'][1]
writing_url

'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2162544'

In [48]:
import io
import requests
import PyPDF2

In [52]:
# Tentativa de extrair texto do pdf da redação final

r = requests.get(writing_url)
f = io.BytesIO(r.content)

reader = PyPDF2.PdfReader(f)
pages = reader.pages
text = "".join([page.extract_text() for page in pages])

PdfReadError: EOF marker not found